In [1]:
# %pip install nltk

In [2]:
import nltk
import re
import string
import pandas as pd
import seaborn as sb
from sklearn.model_selection import train_test_split
from nltk.stem import WordNetLemmatizer

In [3]:
# nltk.download('punkt')
# nltk.download('stopwords')
# nltk.download('wordnet')

In [4]:
def remove_punctuation(text):
  text = text.lower()
  punctuationfree="".join([i for i in text if i not in string.punctuation])
  return punctuationfree
def tokenization(text):
  tokens = re.split(' ',text)
  return tokens
stopwords = nltk.corpus.stopwords.words('english')
def remove_stopwords(text):
  output = []
  for i in text:
    if i in stopwords:
      continue
    output.append(i)
  return output
wordnet_lemmatizer = WordNetLemmatizer()
def lemmatizer(text):
  lemm_text = [wordnet_lemmatizer.lemmatize(word) for word in text]
  return lemm_text
def preprocessing(txt):
    return lemmatizer(remove_stopwords(tokenization(remove_punctuation(txt))))

In [5]:
df = pd.read_csv("C:\StrangerCodes\Jupyter\datasets\spam.csv",\
                 encoding='latin-1')
df.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'],axis=1,inplace=True)
df.rename(columns={'v1':'target', 'v2':'text'}, inplace=True)
df.sample(6)

,target,text
3671,ham,"came to look at the flat, seems ok, in his 50..."
3536,ham,Mmmmm ... It was sooooo good to wake to your w...
2345,ham,"Hi this is yijue, can i meet u at 11 tmr?"
4718,ham,Aiyo... Her lesson so early... I'm still sleep...
5314,spam,Get the official ENGLAND poly ringtone or colo...
3111,ham,Just haven't decided where yet eh ?


In [6]:
for i in range(len(df)):
    df['text'][i] = preprocessing(df['text'][i])

In [7]:
df[:3]

,target,text
0,ham,"[go, jurong, point, crazy, available, bugis, n..."
1,ham,"[ok, lar, joking, wif, u, oni]"
2,spam,"[free, entry, 2, wkly, comp, win, fa, cup, fin..."


In [8]:
x_train, x_test, y_train, y_test = train_test_split(df['text'], df['target'])

In [9]:
p_spam = len(x_train[y_train=='spam'])/len(x_train)
p_ham = len(x_train[y_train=='ham'])/len(x_train)

In [10]:
n_spam = 0
for i in x_train.index:
  if(y_train[i] == 'spam'):
    n_spam += len(x_train[i])
n_ham = 0
for i in x_train.index:
  if(y_train[i] == 'ham'):
    n_ham += len(x_train[i])
n = int(n_ham/n_spam)
ham_index = x_train[y_train=='ham'].index
spam_index = x_train[y_train=='spam'].index
p = {}
q = {}
for i in spam_index:
  for j in x_train[i]:
    try:
      p[j] += 1
    except:
      p[j] = 1
for i in ham_index:
  for j in x_train[i]:
    try:
      q[j] += 1
    except:
      q[j] = 1
p.update((key, value*n) for key, value in p.items())

In [11]:
def check(sample):
  # try:  #custom check
  #   sample = sample.lower()
  #   sample = sample.split()
  # except:
  #   sample
  malt_spam = p_spam
  malt_ham = p_ham
  for i in range(len(sample)):
    valp = p.get(sample[i], n)
    valq = q.get(sample[i], 1)
    malt_spam = malt_spam*valp
    malt_ham = malt_ham*valq
    if(malt_spam>1 or malt_ham>1):
      malt_spam /= 10
      malt_ham /= 10
  if(malt_spam>malt_ham):
    return 'spam', malt_spam, malt_ham
  else:
    return 'ham', malt_spam, malt_ham
def predict(x_test):
  y_pred = []
  for x in x_test:
    y_pred.append(check(x)[0])
  return y_pred
def accuracy(y_test, y_pred):
  c = 0
  y_test = y_test.reset_index()
  y_test.drop(['index'], axis=1, inplace=True)
  for i in range(len(y_pred)):
    if(y_test['target'][i] == y_pred[i]):
      c += 1
  return c/len(y_test)

In [12]:
x_test[y_test=='spam']

875     [shop, till, u, drop, either, 10k, 5k, å£500, ...
1088    [awarded, sipix, digital, camera, call, 090612...
2123    [123, congratulation, , week, competition, dra...
1044    [know, someone, know, fancy, call, 09058097218...
19      [england, v, macedonia, , dont, miss, goalstea...
                              ...                        
12      [urgent, 1, week, free, membership, å£100000, ...
5102    [msg, mobile, content, order, resent, previous...
2108    [free2day, sexy, st, george, day, pic, jordant...
1422    [congratulation, ur, awarded, either, å£500, c...
1021    [guess, somebody, know, secretly, fancy, wanna...
Name: text, Length: 197, dtype: object

In [13]:
check(x_test[12])

('spam', 2577533.6851358684, 4.684058588178989e-10)

In [14]:
y_pred = predict(x_test)
acc = accuracy(y_test, y_pred)*100
print(f'accuracy is {acc:.3f}%')

accuracy is 95.047%


In [15]:
def TP(y_test, y_pred):
  c = 0
  y_test = y_test.reset_index()
  y_test.drop(['index'], axis=1, inplace=True)
  for i in range(len(y_pred)):
    if(y_test['target'][i]=='ham' and y_pred[i]=='ham'):
      c += 1
  return c
def TN(y_test, y_pred):
  c = 0
  y_test = y_test.reset_index()
  y_test.drop(['index'], axis=1, inplace=True)
  for i in range(len(y_pred)):
    if(y_test['target'][i]=='spam' and y_pred[i]=='spam'):
      c += 1
  return c
def FP(y_test, y_pred):
  c = 0
  y_test = y_test.reset_index()
  y_test.drop(['index'], axis=1, inplace=True)
  for i in range(len(y_pred)):
    if(y_test['target'][i]=='spam' and y_pred[i]=='ham'):
      c += 1
  return c
def FN(y_test, y_pred):
  c = 0
  y_test = y_test.reset_index()
  y_test.drop(['index'], axis=1, inplace=True)
  for i in range(len(y_pred)):
    if(y_test['target'][i]=='ham' and y_pred[i]=='spam'):
      c += 1
  return c

In [16]:
tp = TP(y_test, y_pred)
fp = FP(y_test, y_pred)
tn = TN(y_test, y_pred)
fn = FN(y_test, y_pred)

In [17]:
def metrics(tp, tn, fp, fn):
  precision = tp/(tp+fp)
  recall = tp/(tp+fn)
  f1 = (2*precision*recall)/(precision+recall)
  return precision*100, recall*100, f1*100

In [18]:
pc,r,f = metrics(tp, tn, fp, fn)
print(f'the precision is {pc:.3f}%')
print(f'the recall is    {r:.3f}%')
print(f'the f1-score is  {f:.3f}%')

the precision is 99.300%
the recall is    94.900%
the f1-score is  97.050%


Retraining the Model

In [19]:
x = df['text']
y = df['target']
p_spam = len(x[y=='spam'])/len(x)
p_ham = len(x[y=='ham'])/len(x)
n_spam = 0
for i in x.index:
  if(y[i] == 'spam'):
    n_spam += len(x[i])
n_ham = 0
for i in x_train.index:
  if(y[i] == 'ham'):
    n_ham += len(x[i])
n = int(n_ham/n_spam)
ham_index = x[y=='ham'].index
spam_index = x[y=='spam'].index
p = {}
q = {}
for i in spam_index:
  for j in x[i]:
    try:
      p[j] += 1
    except:
      p[j] = 1
for i in ham_index:
  for j in x[i]:
    try:
      q[j] += 1
    except:
      q[j] = 1
p.update((key, value*n) for key, value in p.items())

In [20]:
txt = 'The lottery, a centuries-old tradition, symbolizes the tantalizing allure of fate. In the hearts of participants, it ignites dreams of prosperity and a life transformed. Yet, beneath this veneer of excitement lies a tapestry of controversy and ethical debate. At its essence, the lottery represents humanity\'s eternal optimism. It thrives on the hope that a small investment could yield a life-altering windfall, offering a glimpse into a world free from financial constraints. In every draw, individuals hold their breath, imagining the possibilities that winning numbers might bring. However, this hope is not universal. Critics argue that lotteries exploit the vulnerable, enticing them with dreams while siphoning away precious resources. For those facing economic hardship, the temptation to invest in lottery tickets can lead to financial instability, exacerbating societal inequalities. Psychologically, the lottery\'s grip on the human mind is profound. It triggers a unique blend of hope and desperation, driving people to spend beyond their means in pursuit of the elusive jackpot. This phenomenon, known as "lottery fever," underscores the power of the lottery to blur the lines between rationality and desire. Ethically, lotteries raise complex questions. While they contribute substantially to public funds, critics question the morality of financing essential services through a game of chance. The tension between individual freedom and collective well-being forms the core of this debate, forcing society to weigh the consequences of indulging in this age-old pastime. In an increasingly digital world, lotteries have adapted, finding new avenues to entice participants. Online platforms have transformed the way people engage with the game, expanding its reach and influence. The debate surrounding its societal impact, however, remains as relevant as ever. In conclusion, the lottery embodies the duality of human nature: our dreams of a better life juxtaposed against the ethical quandaries of exploiting those dreams. Its enduring popularity underscores our collective fascination with fate and the delicate balance between personal ambition and social responsibility. As the debate rages on, the lottery stands as a poignant reminder of our shared hopes, vulnerabilities, and the complexities of chance that define the human experience.'

In [21]:
txt = txt.split('. ')[:len(txt.split('. '))-1]
txt

['The lottery, a centuries-old tradition, symbolizes the tantalizing allure of fate',
 'In the hearts of participants, it ignites dreams of prosperity and a life transformed',
 'Yet, beneath this veneer of excitement lies a tapestry of controversy and ethical debate',
 "At its essence, the lottery represents humanity's eternal optimism",
 'It thrives on the hope that a small investment could yield a life-altering windfall, offering a glimpse into a world free from financial constraints',
 'In every draw, individuals hold their breath, imagining the possibilities that winning numbers might bring',
 'However, this hope is not universal',
 'Critics argue that lotteries exploit the vulnerable, enticing them with dreams while siphoning away precious resources',
 'For those facing economic hardship, the temptation to invest in lottery tickets can lead to financial instability, exacerbating societal inequalities',
 "Psychologically, the lottery's grip on the human mind is profound",
 'It trig

In [22]:
txtt = []
for i in range(len(txt)):
    txtt.append(preprocessing(txt[i]))
txtt[:2]

[['lottery',
  'centuriesold',
  'tradition',
  'symbolizes',
  'tantalizing',
  'allure',
  'fate'],
 ['heart',
  'participant',
  'ignites',
  'dream',
  'prosperity',
  'life',
  'transformed']]

In [23]:
def spamcheck(txtt):
    p = 0
    q = 0
    pdict = {}
    qdict = {}
    for i in range(len(txtt)):
        c,a,b = check(txtt[i])
        if c == 'spam':
            p+=a
            pdict[i] = a
        else:
            q+=b
            qdict[i] = b
    pdict = dict(sorted(pdict.items(), key = lambda x:x[1], reverse=True))
    qdict = dict(sorted(qdict.items(), key = lambda x:x[1], reverse=True))
    if (p>q):
        return 'spammy', pdict, qdict
    elif (p<q):
        return 'hammy', pdict, qdict
    else:
        return '(:'

In [24]:
res, pdict, qdict = spamcheck(txtt)

In [25]:
print('The text is', res)

The text is spammy


In [26]:
print('the top 3 spammy sentences are:')
for i in list(pdict.keys())[:3]:
    print(txt[i])

the top 3 spammy sentences are:
While they contribute substantially to public funds, critics question the morality of financing essential services through a game of chance
At its essence, the lottery represents humanity's eternal optimism
The tension between individual freedom and collective well-being forms the core of this debate, forcing society to weigh the consequences of indulging in this age-old pastime


In [27]:
print('the top three hammy sentences are:')
for i in list(qdict.keys())[:3]:
    print(txt[i])

the top three hammy sentences are:
In every draw, individuals hold their breath, imagining the possibilities that winning numbers might bring
In conclusion, the lottery embodies the duality of human nature: our dreams of a better life juxtaposed against the ethical quandaries of exploiting those dreams
Ethically, lotteries raise complex questions
